# Ejercicios propuestos: *Input*/*Output* + Serialización + RegEx

Los siguientes problemas se dejan como opción para ejercitar los conceptos revisados en la segunda parte sobre I/O (semana 11). Si tienes dudas sobre algún problema o alguna solución, no dudes en dejar una issue en el foro del curso.

## Ejercicio 1

El archivo **simpatico.bmp** ha sido corrompido con bytes no correspondientes. Para limpiarlo, deberá realizar el siguiente algoritmo:

- Abra el archivo **malvado.mal** en modo lectura de bytes.
- Obtenga chunks de 8 bytes.
- Encuentre el máximo de esos 8 bytes.
- Elimine todos los bytes con ese valor.
- Repita el proceso con el resto de los bytes.
- Escriba el archivo **simpatico.bmp** con los bytes obtenidos.

Es decir,

Si tengo 8 bytes con los siguientes valores:

4 8 255 15 16 23 255 42

Obtenemos el máximo de ese chunk de bytes: 255

Elimino todos los bytes con ese valor, por lo que los que se han de escribir serán:

4 8 15 16 23 42

In [8]:
#Abro el archivo
with open('malvado.mal', 'rb') as bytefile:
    byte_read = bytefile.read()
    byte_out = bytearray()
    for i in range(0, len(byte_read), 8):
        #obtengo chunks de 8 bytes
        chunk = bytearray(byte_read[i:i+8])
        #obtengo el maximo
        maximum = max(chunk)
        #elimino todos los bytes de ese valor
        while maximum in chunk:
            chunk.remove(maximum)
        byte_out += chunk
#escribo el archivo
with open('simpatico.bmp', 'wb') as file_out:
    file_out.write(byte_out)

## Ejercicio 2

Han contaminado el recetario de las pizzas del Tini Tamburini, por lo cual ha venido a conseguir ayuda mediante este ejercicio propuesto.

Deberá deserializar el archivo **pizzeria.json** que contiene una lista de objetos de clase Pizza.

Estos objetos tienen el atributo **ingredientes** el cual ha sido contaminado con elementos que no son comestibles, para identificarlos, el Tini le entrega un archivo llamado **comestibles.json**, el cual le ayudará a filtrar los ingredientes mediante un **object_hook**.

Finalmente, imprima todas las pizzas mediante un **format string**.

In [11]:
import json


class Pizza:
    def __init__(self, nombre, ingredientes, *args, **kwargs):
        self.nombre = nombre
        self.ingredientes = ingredientes
    def __repr__(self):
        return self.nombre
    def __str__(self):
        return 'Pizza: {} \nIngredientes:{}'.format(self.nombre, self.ingredientes)

def filtrar_comestibles(diccionario):
    with open('comestibles.json') as comestibles_file:
        comestible = json.load(comestibles_file)
    diccionario['ingredientes'] = list(filter(lambda x: x in comestible, 
                                              diccionario['ingredientes']))
    return Pizza(**diccionario)

with open('pizzeria.json') as json_file:
    pizzas = json.load(json_file, object_hook=filtrar_comestibles)
    
    
for p in pizzas:
    print(p)

Pizza: hawaiana 
Ingredientes:['pinia', 'queso', 'jamon']
Pizza: vegetariana 
Ingredientes:['queso', 'aceitunas', 'champiniones']
Pizza: machas a la parmesana 
Ingredientes:['machas', 'queso']
Pizza: pepperoni 
Ingredientes:['pepperoni', 'queso']
Pizza: pollo bbq 
Ingredientes:['cebolla', 'queso', 'pollo bbq']
Pizza: jamon palmito 
Ingredientes:['jamon', 'palmitos', 'queso']
Pizza: white 
Ingredientes:['aceitunas', 'tomate']
Pizza: napolitana 
Ingredientes:['queso', 'albahaca', 'tomate']
Pizza: queso 
Ingredientes:['queso', 'queso']


## Ejercicio 3

El profesor Vicente es un muy buen baterista, y un día le llego una canción de un alumno escrita de una manera muy peculiar:

BUMpatapatapatatss;-;BUMBUMpatatsspatatss;-;BUMpataBUMpataBUM;-;BUMtsstssBUMpata...

El ingeniosamente, descubrió que significa cada elemento:

- BUM: Bombo -> O
- pata: Caja -> ././
- tss: Platillo  -> x/

Mientras que los ';-;' sería una barra de compás " | "

Como favor, te pide que crees una función **traducir_pata(cancion)** que use Regex para reemplazar los elementos de la canción por la nomenclatura que te presenta.

Ejemplo

BUMpatapatapatatss;-;BUMBUMpatatsspatatss;-;BUMpataBUMpataBUM;-;BUMtsstssBUMpata

retornaría

O ././ ././ ././ x/ | O O ././ x/ ././ x/ | O ././ O ././ O | O x/ x/ O ././ 

In [19]:
import re

def traducir_pata(cancion):
    cancion = re.sub("BUM", "O ", cancion)
    cancion = re.sub("pata", "././ ", cancion)
    cancion = re.sub("tss", "x/ ", cancion)
    cancion = re.sub(";-;", '| ', cancion)
    return cancion

print(traducir_pata("BUMpatapatapatatss;-;BUMBUMpatatsspatatss;-;BUMpataBUMpataBUM;-;BUMtsstssBUMpata"))

O ././ ././ ././ x/ | O O ././ x/ ././ x/ | O ././ O ././ O | O x/ x/ O ././ 


## Ejercicio 4

En las descripciones de un conjunto de obras de arte se encuentra un mensaje oculto, por lo que deberás desifrar este mensaje a través de Serializacion Pickle.

Para lograr esto, deberás implementar las siguientes funciones:

cargar obras(ruta_obras): El archivo operas.json contiene toda la información de las obras.
Las obras contienen algunos datos que no son relevantes para la solución de este misterio. Por esto,
se te entrega el archivo caratteristicas.json 5 con las caracterı́sticas que sı́ deberás obtener de
las obras, mediante la función obras hook. Esta función deberá retornar una lista de objetos de la
clase Obra

generar mensaje(lista_obras): Tu función debe serializar cada una de las obras utilizando pickle. Durante este proceso deberás agregar, a cada obra, el atributo messaggio con un mensaje generado por 3 palabras al azar del atributo descrizione. Los archivos generados deben ser guardados con el formato de nombre < nome >-< autore >.opera, donde nome y autore son el nombre de la
obra y el nombre de su respectivo autor, en la carpeta Obras.

In [ ]:
import json
import os
import pickle
import random

CARATTERISTICAS_CAMMINO = 'caratteristicas.json'
OPERAS_CAMMINO = 'operas.json'

class Obra:
    """
    Clase para las Obras de Antonini Da Ossa
    """

    def __init__(self, nome=None, autore=None, anno=None, posto=None,
                 stile=None, descrizione=None):

        self.nome = nome
        self.autore = autore
        self.anno = anno
        self.posto = posto
        self.stile = stile
        self.descrizione = descrizione

    def __getstate__(self):
        """
        Serializa las obras, agregando el atributo 'messaggio' al diccionario.
        """

        pass


def cargar_obras(ruta_obras):
    """

    Funcion que carga las obras con las características pedidas,
    y luego entrega la lista de Obras.
    :param ruta_obras: Path de archivo de obras a cargar

    NOTA: DEBE USARSE la funcion obras_hook() como
     object hook para hacer el filtrado

    """
    pass


def obras_hook(dict_obras):
    """
    Object hook que hace los objetos de la clase Obra y los añade a una lista.
    HINT: Utilizar aquí el archivo caratteristicas ;)
    """

    pass


def generar_mensaje(lista_obras):
    """
    Serializa las obras y las guarda en archivos
    :param lista_obras: Lista de objetos de tipo obra cargados.
    """

    pass
